# Libraries

In [3]:
# libraries
# ----------
#
#%pip install wget

import pandas as pd
import wget

ModuleNotFoundError: No module named 'wget'

# Downloading data

In [5]:
# remove existing files
! rm *.csv

# urls of the files
urls = ['https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv', 
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv',
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv']

# download files
for url in urls:
    filename = wget.download(url)

# Dataframes

In [5]:
# dataset
# --------

conf_df = pd.read_csv('../datasets/time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('../datasets/time_series_covid19_deaths_global.csv')
recv_df = pd.read_csv('../datasets/time_series_covid19_recovered_global.csv')

In [6]:
# conf_df.head()
#deaths_df.head()
# recv_df.head()

In [7]:
conf_df.columns
# deaths_df.columns
# recv_df.columns

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '4/28/20', '4/29/20', '4/30/20', '5/1/20', '5/2/20', '5/3/20', '5/4/20',
       '5/5/20', '5/6/20', '5/7/20'],
      dtype='object', length=111)

In [8]:
conf_df.columns[4:]

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20',
       ...
       '4/28/20', '4/29/20', '4/30/20', '5/1/20', '5/2/20', '5/3/20', '5/4/20',
       '5/5/20', '5/6/20', '5/7/20'],
      dtype='object', length=107)

# Merging dataframes

In [9]:
dates = conf_df.columns[4:]

conf_df_long = conf_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Confirmed')

deaths_df_long = deaths_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Deaths')

recv_df_long = recv_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Recovered')

recv_df_long = recv_df_long[recv_df_long['Country/Region']!='Canada']

print(conf_df_long.shape)
print(deaths_df_long.shape)
print(recv_df_long.shape)

(28462, 6)
(28462, 6)
(26857, 6)


In [10]:
# full_table = pd.concat([conf_df_long, deaths_df_long['Deaths'], recv_df_long['Recovered']], 
#                        axis=1, sort=False)

full_table = pd.merge(left=conf_df_long, right=deaths_df_long, how='left',
                      on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])
full_table = pd.merge(left=full_table, right=recv_df_long, how='left',
                      on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])

full_table.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.0000,65.0000,1/22/20,0,0,0.0
1,NaN,Albania,41.1533,20.1683,1/22/20,0,0,0.0
2,NaN,Algeria,28.0339,1.6596,1/22/20,0,0,0.0
3,NaN,Andorra,42.5063,1.5218,1/22/20,0,0,0.0
4,NaN,Angola,-11.2027,17.8739,1/22/20,0,0,0.0


In [11]:
full_table.shape

(28462, 8)

In [12]:
full_table.isna().sum()

Province/State    19688
Country/Region        0
Lat                   0
Long                  0
Date                  0
Confirmed             0
Deaths                0
Recovered          1926
dtype: int64

In [13]:
full_table[full_table['Recovered'].isna()]['Country/Region'].value_counts()

Canada         1605
Mozambique      107
Timor-Leste     107
Syria           107
Name: Country/Region, dtype: int64

In [14]:
full_table[full_table['Recovered'].isna()]['Date'].value_counts()

2/20/20    18
3/14/20    18
4/22/20    18
2/24/20    18
5/2/20     18
           ..
4/20/20    18
2/15/20    18
2/13/20    18
2/1/20     18
4/18/20    18
Name: Date, Length: 107, dtype: int64

In [15]:
full_table['Recovered'] = full_table['Recovered'].fillna(0)
full_table['Recovered'] = full_table['Recovered'].astype('int')
full_table.isna().sum()

Province/State    19688
Country/Region        0
Lat                   0
Long                  0
Date                  0
Confirmed             0
Deaths                0
Recovered             0
dtype: int64

# Preprocessing

In [16]:
# renaming
# ========

# renaming countries, regions, provinces
full_table['Country/Region'] = full_table['Country/Region'].replace('Korea, South', 'South Korea')

In [17]:
# removing
# =======

# removing canada's recovered values
full_table = full_table[full_table['Province/State'].str.contains('Recovered')!=True]

# removing county wise data to avoid double counting
full_table = full_table[full_table['Province/State'].str.contains(',')!=True]

# Fixing off data

In [18]:
# new values
feb_12_conf = {'Hubei' : 34874}

In [19]:
# function to change value
def change_val(date, ref_col, val_col, dtnry):
    for key, val in dtnry.items():
        full_table.loc[(full_table['Date']==date) & (full_table[ref_col]==key), val_col] = val

In [20]:
# changing values
change_val('2/12/20', 'Province/State', 'Confirmed', feb_12_conf)

In [21]:
# checking values
full_table[(full_table['Date']=='2/12/20') & (full_table['Province/State']=='Hubei')]

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
5648,Hubei,China,30.9756,112.2707,2/12/20,34874,1068,2686


# Saving final data

In [22]:
full_table.to_csv('covid_19_clean_complete.csv', index=False)

# Country wise data

In [23]:
# china
# =====

china_province_wise = full_table[full_table['Country/Region']=='China']
china_province_wise['Province/State'].unique()
china_province_wise.to_csv('china_province_wise.csv', index=False)

In [24]:
# Australia
# =========

australia_state_wise = full_table[full_table['Country/Region']=='Australia']
australia_state_wise['Province/State'].unique()
australia_state_wise.to_csv('australia_state_wise.csv', index=False)

In [25]:
# Canada
# ======

#canada_state_wise = full_table[full_table['Country/Region']=='Canada']
#canada_state_wise['Province/State'].unique()
#canada_state_wise.to_csv('canada_state_wise.csv', index=False)

In [26]:
# Malaysia
# ======

malaysia_state_wise = full_table[full_table['Country/Region']=='Malaysia']
malaysia_state_wise['Province/State'].unique()
malaysia_state_wise.to_csv('malaysia_state_wise.csv', index=False)

In [27]:
# Singapore
# ======

singapore_state_wise = full_table[full_table['Country/Region']=='Singapore']
singapore_state_wise['Province/State'].unique()
singapore_state_wise.to_csv('singapore_state_wise.csv', index=False)

In [28]:
# USA
# ======

us_state_wise = full_table[full_table['Country/Region']=='USA']
us_state_wise['Province/State'].unique()
us_state_wise.to_csv('usa_state_wise.csv', index=False)

In [32]:
list_of_values = ['USA','Singapore']
sin_us_state_wise = full_table[full_table['Country/Region'].isin(list_of_values)]
sin_us_state_wise.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
196,NaN,Singapore,1.2833,103.8333,1/22/20,0,0,0
462,NaN,Singapore,1.2833,103.8333,1/23/20,1,0,0
728,NaN,Singapore,1.2833,103.8333,1/24/20,3,0,0
994,NaN,Singapore,1.2833,103.8333,1/25/20,3,0,0
1260,NaN,Singapore,1.2833,103.8333,1/26/20,4,0,0


# USA data

In [27]:
# urls of the files
urls = ['https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv', 
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv']

# download files
for url in urls:
    filename = wget.download(url)

In [28]:
us_conf_df = pd.read_csv('time_series_covid19_confirmed_US.csv')
us_deaths_df = pd.read_csv('time_series_covid19_deaths_US.csv')

In [29]:
# us_conf_df.head()
# us_deaths_df.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,141,141,145,145,145,145,145,145,149,149
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,14,14,14,14,14,14,14,14,15,15
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,1400,1433,1539,1575,1757,1808,1843,1924,1968,2031
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,57,57,66,66,66,66,66,66,66,66


In [27]:
# us_conf_df.columns
# us_deaths_df.columns

In [30]:
ids = us_conf_df.columns[0:11]
us_dates = us_conf_df.columns[11:]

us_conf_df_long = us_conf_df.melt(id_vars=ids, value_vars=us_dates, var_name='Date', value_name='Confirmed')
us_deaths_df_long = us_deaths_df.melt(id_vars=ids, value_vars=us_dates, var_name='Date', value_name='Deaths')

In [31]:
us_conf_df_long.head()
# us_deaths_df_long.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Date,Confirmed
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,"American Samoa, US",1/22/20,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,"Guam, US",1/22/20,0
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,"Northern Mariana Islands, US",1/22/20,0
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,"Puerto Rico, US",1/22/20,0
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,"Virgin Islands, US",1/22/20,0


In [32]:
print(us_conf_df_long.shape)
print(us_deaths_df_long.shape)

(348927, 13)
(348927, 13)


In [33]:
ft_ids = us_conf_df_long.columns[:-1]
ft_ids

Index(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Date'],
      dtype='object')

In [34]:
us_full_table = pd.concat([us_conf_df_long, us_deaths_df_long[['Deaths']]], axis=1)
us_full_table.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Date,Confirmed,Deaths
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,"American Samoa, US",1/22/20,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,"Guam, US",1/22/20,0,0
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,"Northern Mariana Islands, US",1/22/20,0,0
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,"Puerto Rico, US",1/22/20,0,0
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,"Virgin Islands, US",1/22/20,0,0


In [35]:
us_full_table.to_csv('usa_county_wise.csv', index=False)